To start, upload:
- `.tsv.lrz` file containing data from most recent scrape
- `data.py`
- `model.py`
- `requirements.txt`

In [ ]:
!apt-get install lrzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  lrzip
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 234 kB of archives.
After this operation, 1,152 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 lrzip amd64 0.631-1 [234 kB]
Fetched 234 kB in 1s (415 kB/s)
Selecting previously unselected package lrzip.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../lrzip_0.631-1_amd64.deb ...
Unpacking lrzip (0.631-1) ...
Setting up lrzip (0.631-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [ ]:
!mkdir data/
!lrunzip -cdivv 11-25-20_21-1500.tsv.lrz -o data/meme_data.tsv

In [ ]:
!ls -sh

total 23M
 23M 11-25-20_21-1500.tsv.lrz  4.0K model.py	 4.0K requirements.txt
4.0K data.py		       4.0K __pycache__  4.0K sample_data


In [ ]:
!pip3 install -r requirements.txt

In [ ]:
import data

In [ ]:
dataset = data.MemesDataModule()
dataset.prepare_data() # this generates a pickle
dataset.setup() # generates dataloaders

In [ ]:
train_dl = dataset.train_dataloader()
val_dl = dataset.val_dataloader()
test_dl = dataset.test_dataloader()

In [ ]:
from model import GPT2

In [ ]:
GPT2_model = GPT2(0.02, 1, 1)

Some weights of GPT2DoubleHeadsModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'multiple_choice_head.summary.weight', 'lm_head.weight', 'multiple_choice_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
GPT2_model.configure_optimizers()

([AdamW (
  Parameter Group 0
      amsgrad: False
      betas: (0.9, 0.999)
      eps: 1e-08
      initial_lr: 0.02
      lr: 0.0
      weight_decay: 0.0
  )], [<torch.optim.lr_scheduler.LambdaLR at 0x7fa46bca29b0>])

In [ ]:
one_item = next(iter(train_dl))['caption']
GPT2_model.forward(one_item)

GPT2DoubleHeadsModelOutput([('logits',
                             tensor([[[ -31.7896,  -30.7168,  -32.1722,  ...,  -39.5315,  -39.7643,
                                        -31.6573],
                                      [ -63.3437,  -60.9143,  -61.4364,  ...,  -72.3313,  -72.0760,
                                        -62.4938],
                                      [ -60.3880,  -60.5608,  -59.7230,  ...,  -72.2708,  -71.4771,
                                        -62.5310],
                                      ...,
                                      [ -97.0929,  -99.5411, -102.2409,  ..., -114.4586, -113.9481,
                                        -99.7832],
                                      [-114.9237, -116.6466, -114.1535,  ..., -127.3505, -127.2874,
                                       -111.7825],
                                      [ -86.8066,  -80.1242,  -82.1678,  ..., -100.1581, -100.7708,
                                        -88.2601]]], grad_fn=<U

In [ ]:
GPT2_model.train_step(one_item, _)
GPT2_model.val_step(one_item, _)
GPT2_model.test_step(one_item, _)

The following code blocks test the capability of the model on distributed systems (they should all perform successfully).

In [ ]:
import pickle
with open('test.pickle', 'wb') as handle:
  pickle.dump(GPT2_model, handle)

In [17]:
with open('test2.pickle', 'wb') as handle:
  pickle.dump(train_dl, handle)
with open('test3.pickle', 'wb') as handle:
  pickle.dump(val_dl, handle)
with open('test4.pickle', 'wb') as handle:
  pickle.dump(test_dl, handle)